<a href="https://colab.research.google.com/github/luiz-oliveir/lstm_mensal/blob/main/LSTM_VAE_Mensal_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/luiz-oliveir/LSTM_mensal.git

Cloning into 'LSTM_mensal'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 86 (delta 11), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (86/86), 14.20 MiB | 15.58 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [6]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import glob
import datetime
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import logging
import pickle

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('lstm_vae_mensal.log'),
        logging.StreamHandler()
    ]
)

# Configurações
row_mark = 740
batch_size = 128
timesteps = 7  # Janela de tempo para análise
n_features = 1  # Número de features (temperatura)
latent_dim = 32  # Dimensão do espaço latente
epoch_num = 100
threshold = None

# Diretórios
base_dir = os.path.abspath(os.path.dirname(''))
data_dir = r"C:\Users\Augusto-PC\Documents\GitHub\LSTM\Convencionais processadas temperaturas"  # Correct path
model_dir = os.path.join(base_dir, "lstm_vae_model")
images_dir = os.path.join(base_dir, "lstm_vae_images")
results_dir = os.path.join(base_dir, "Resumo resultados")

# Verificar e criar diretórios
for dir_path in [model_dir, images_dir, results_dir]:
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        logging.info(f"Directory created: {dir_path}")
    else:
        logging.info(f"Directory verified: {dir_path}")

# Dicionário de meses
meses = {
    1:'jan', 2:'fev', 3:'mar', 4:'abr', 5:'mai', 6:'jun',
    7:'jul', 8:'ago', 9:'set', 10:'out', 11:'nov', 12:'dez'
}

class ReparameterizationLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(ReparameterizationLayer, self).__init__(**kwargs)

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch_size = tf.shape(z_mean)[0]
        latent_dim = tf.shape(z_mean)[1]
        epsilon = tf.random.normal(shape=(batch_size, latent_dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

class RepeatVectorLayer(tf.keras.layers.Layer):
    def __init__(self, timesteps, **kwargs):
        super(RepeatVectorLayer, self).__init__(**kwargs)
        self.timesteps = timesteps

    def call(self, inputs):
        return tf.repeat(tf.expand_dims(inputs, axis=1), repeats=self.timesteps, axis=1)

    def get_config(self):
        config = super(RepeatVectorLayer, self).get_config()
        config.update({'timesteps': self.timesteps})
        return config

class TemperatureWeightLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(TemperatureWeightLayer, self).__init__(**kwargs)

    def call(self, x):
        mean = tf.keras.backend.mean(x, axis=[1, 2], keepdims=True)
        std = tf.keras.backend.std(x, axis=[1, 2], keepdims=True) + tf.keras.backend.epsilon()
        z_scores = tf.abs((x - mean) / std)
        weights = tf.exp(z_scores)
        weights = weights / (tf.keras.backend.mean(weights, axis=[1, 2], keepdims=True) + tf.keras.backend.epsilon())
        return weights

class LSTM_VAE(tf.keras.Model):
    def __init__(self, timesteps=7, n_features=1, latent_dim=32, **kwargs):
        super(LSTM_VAE, self).__init__(**kwargs)
        self.timesteps = timesteps
        self.n_features = n_features
        self.latent_dim = latent_dim

        # First LSTM layer
        self.lstm = tf.keras.layers.LSTM(32, return_sequences=True, name='lstm')

        # First LSTM output processing
        self.dense = tf.keras.layers.Dense(4, name='dense')
        self.dense_1 = tf.keras.layers.Dense(4, name='dense_1')
        self.dense_2 = tf.keras.layers.Dense(32, name='dense_2')

        # Second LSTM layer
        self.lstm_1 = tf.keras.layers.LSTM(32, return_sequences=True, name='lstm_1')

        # Second LSTM output processing
        self.dense_3 = tf.keras.layers.Dense(32, name='dense_3')
        self.dense_4 = tf.keras.layers.Dense(16, name='dense_4')
        self.dense_5 = tf.keras.layers.Dense(1, name='dense_5')

        # Final processing branch
        self.dense_6 = tf.keras.layers.Dense(16, name='dense_6')
        self.dense_7 = tf.keras.layers.Dense(1, name='dense_7')

        # Additional layers
        self.repeat_vector = tf.keras.layers.RepeatVector(timesteps, name='repeat_vector')
        self.dropout = tf.keras.layers.Dropout(0.2, name='dropout')
        self.layer_norm = tf.keras.layers.LayerNormalization(name='layer_normalization')

    def call(self, inputs, training=None):
        # First LSTM processing
        x = self.lstm(inputs)

        # Process first LSTM output
        x = self.dense(x)
        x = self.dense_1(x)
        x = self.dense_2(x)

        # Second LSTM processing
        x = self.lstm_1(x)

        # Get last timestep for repeat vector
        last_timestep = x[:, -1, :]
        x = self.repeat_vector(last_timestep)

        # Process repeated vector
        x = self.dense_3(x)
        x = self.dense_4(x)

        # Apply dropout during training
        if training:
            x = self.dropout(x)

        # Apply layer normalization
        x = self.layer_norm(x)

        # Generate outputs through two branches
        output1 = self.dense_5(x)
        x = self.dense_6(x)
        output2 = self.dense_7(x)

        # Combine outputs
        outputs = tf.concat([output1, output2], axis=-1)

        return outputs

    def get_config(self):
        config = super(LSTM_VAE, self).get_config()
        config.update({
            'timesteps': self.timesteps,
            'n_features': self.n_features,
            'latent_dim': self.latent_dim
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

def reshape(da):
    """Reshape dados para formato LSTM"""
    data = []
    for i in range(len(da) - timesteps + 1):
        data.append(da[i:(i + timesteps)])
    return np.array(data)

def prepare_training_data(data, batch_size=128):
    """Prepara dados para o modelo"""
    data = reshape(data)
    data = data.reshape(-1, timesteps, n_features)
    dataset = tf.data.Dataset.from_tensor_slices(data)
    dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)
    return dataset

def calculate_advanced_metrics(predictions, originals):
    """Calculate advanced evaluation metrics"""
    mse = np.mean(np.square(predictions - originals))
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(predictions - originals))
    mape = np.mean(np.abs((originals - predictions) / originals)) * 100

    # Calculate log likelihood metrics
    residuals = predictions - originals
    std = np.std(residuals)
    log_likelihood = -0.5 * np.log(2 * np.pi * std**2) - 0.5 * (residuals**2) / (std**2)
    mean_log_likelihood = np.mean(log_likelihood)

    # Calculate percentile analysis
    percentiles = [1, 5, 25, 50, 75, 95, 99]
    orig_percentiles = np.percentile(originals, percentiles)
    pred_percentiles = np.percentile(predictions, percentiles)

    percentile_errors = {
        f'p{p}_error': abs(o - p)
        for p, o, p in zip(percentiles, orig_percentiles, pred_percentiles)
    }

    metrics = {
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'mape': mape,
        'mean_log_likelihood': mean_log_likelihood,
        **percentile_errors
    }

    return metrics

def train_monthly_models(excel_files):
    """Train separate LSTM VAE models for each month"""
    for file_path in excel_files:
        logging.info(f"Processing file: {file_path}")

        try:
            # Read data
            df = pd.read_excel(file_path)
            df['Data'] = pd.to_datetime(df['Data'])
            df.set_index('Data', inplace=True)

            # Sort index to ensure chronological order
            df = df.sort_index()

            # Process each month
            for month in range(1, 13):
                month_data = df[df.index.month == month]

                if len(month_data) < batch_size:
                    logging.warning(f"Insufficient data for month {meses[month]} in {os.path.basename(file_path)}")
                    continue

                logging.info(f"Training model for month: {meses[month]} from file {os.path.basename(file_path)}")

                # Scale data
                scaler = MinMaxScaler()
                scaled_data = scaler.fit_transform(month_data.values.reshape(-1, 1))

                # Save scaler
                scaler_path = os.path.join(model_dir, f'scaler_{meses[month]}.pkl')
                with open(scaler_path, 'wb') as f:
                    pickle.dump(scaler, f)

                # Prepare training data
                train_dataset = prepare_training_data(scaled_data, batch_size)

                # Create and compile model
                model = LSTM_VAE(timesteps=timesteps, n_features=n_features, latent_dim=latent_dim)

                # Build model with input shape
                dummy_input = tf.zeros((1, timesteps, n_features))
                _ = model(dummy_input, training=False)

                # Compile model
                model.compile(optimizer='adam', loss='mse')

                # Add callbacks
                callbacks = [
                    tf.keras.callbacks.EarlyStopping(
                        monitor='val_loss',
                        patience=10,
                        restore_best_weights=True
                    ),
                    tf.keras.callbacks.ReduceLROnPlateau(
                        monitor='val_loss',
                        factor=0.5,
                        patience=5,
                        min_lr=1e-6
                    )
                ]

                # Train model
                history = model.fit(
                    train_dataset,
                    epochs=epoch_num,
                    validation_split=0.2,
                    callbacks=callbacks,
                    verbose=1
                )

                # Save model
                model_path = os.path.join(model_dir, f'lstm_vae_model_{meses[month]}.h5')
                model.save(model_path)

                # Plot and save training history
                plt.figure(figsize=(10, 6))
                plt.plot(history.history['loss'], label='Training Loss')
                plt.plot(history.history['val_loss'], label='Validation Loss')
                plt.title(f'Training History - Month: {meses[month]}')
                plt.xlabel('Epoch')
                plt.ylabel('Loss')
                plt.legend()
                plt.savefig(os.path.join(images_dir, f'training_history_{meses[month]}.png'))
                plt.close()

                logging.info(f"Model for month {meses[month]} saved successfully")

        except Exception as e:
            logging.error(f"Error processing file {file_path}: {str(e)}")
            continue

def main():
    # Find all Excel files in the data directory
    excel_files = glob.glob(os.path.join(data_dir, "*.xlsx"))

    if not excel_files:
        logging.error(f"No Excel files found in {data_dir}")
        return

    logging.info(f"Found {len(excel_files)} Excel files in {data_dir}")

    try:
        train_monthly_models(excel_files)
        logging.info("Training completed successfully")
    except Exception as e:
        logging.error(f"Error during training: {str(e)}")
        raise

if __name__ == "__main__":
    main()

ERROR:root:No Excel files found in /Convencionais processadas temperaturas - Copia
